# continue doing train test split,preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%run /Users/yaoyao/Documents/datascience/toolbox/toolbox1_dataview.ipynb
%matplotlib inline

## load processed data

In [2]:
with open('raw_y.csv') as f:
    raw_y = pd.read_csv(f)

In [3]:
with open('train_ready.csv') as f:
    train_ready = pd.read_csv(f)

In [6]:
with open('test.csv') as f:
    raw_test = pd.read_csv(f)

In [8]:
raw_test.head()
train_ready.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,34ba1a28d02ba8ba,2019-08-06T04:57:18.904Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,4b57c9a59474a1b9,2019-08-06T04:57:45.301Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,2b9d5af79bcdb79f,2019-08-06T04:58:14.538Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,2b9d5af79bcdb79f,2019-08-06T04:58:14.615Z,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:16.680Z,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [9]:
raw_y.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,f56e0afc,a9ef3ecb3d1acc6a,2019-08-06T20:49:59.095Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Bird Measurer (Assessment),Assessment,TREETOPCITY
1,ec138c1c,a9ef3ecb3d1acc6a,2019-08-06T20:49:59.118Z,"{""stage_number"":1,""event_count"":2,""game_time"":...",0006a69f,2,2020,0,Bird Measurer (Assessment),Assessment,TREETOPCITY
2,1375ccb7,a9ef3ecb3d1acc6a,2019-08-06T20:49:59.121Z,"{""description"":""Use the caterpillars to measur...",0006a69f,3,3010,0,Bird Measurer (Assessment),Assessment,TREETOPCITY
3,51102b85,a9ef3ecb3d1acc6a,2019-08-06T20:50:01.686Z,"{""hat"":0,""caterpillar"":""middle"",""coordinates"":...",0006a69f,4,4030,2616,Bird Measurer (Assessment),Assessment,TREETOPCITY
4,4a4c3d21,a9ef3ecb3d1acc6a,2019-08-06T20:50:03.032Z,"{""height"":8,""bird_height"":8,""correct"":true,""ca...",0006a69f,5,4025,3974,Bird Measurer (Assessment),Assessment,TREETOPCITY


In [107]:
def get_accuracy(raw_y):
    # note here, if a session didn't reach the end, we don't have the session here. need to addback in the end
    results = raw_y[(raw_y['title'] == 'Bird Measurer (Assessment)') & (raw_y['event_code'] == 4110)]
    results = results.append(raw_y[(raw_y['title'] != 'Bird Measurer (Assessment)') & (raw_y['event_code'] == 4100)])
    atmp = results.event_data.str.extract(r'("correct"):(\w+)')
    results.insert(6,'attempts',atmp.loc[:,1])
    mapping={"true":1,"false":0}
    results['atmp_binary'] = results['attempts'].map(mapping)
    opt = results.groupby('installation_id').mean()
    opt['acgroup'] = 1
    opt.loc[opt[abs(opt['atmp_binary'] - 0.5) < 1e-9].index,'acgroup'] = 2
    opt.loc[opt[abs(opt['atmp_binary'] - 1) < 1e-9].index,'acgroup'] = 3
    opt.loc[opt[abs(opt['atmp_binary'] - 0) < 1e-9].index,'acgroup'] = 0
    
    #build y
    opty = pd.merge(pd.DataFrame(index = raw_y['installation_id'].unique()).reset_index(),
                    opt.acgroup,how='outer',left_on='index',right_on='installation_id').fillna(0)
    opty = opty.rename(columns={'index':'installation_id'})
    opty['acgroup'] = opty.acgroup.astype(int)
    
    return results, opt, opty





In [143]:
y_ready,opt, opty = get_accuracy(raw_y)

In [202]:
def get_acgroup(records):
    
    #this code is used to get accuracy group from the raw records
    results = records[(records['title'] == 'Bird Measurer (Assessment)') & (records['event_code'] == 4110)]
    results = results.append(records[(records['title'] != 'Bird Measurer (Assessment)') & (records['event_code'] == 4100)])
    atmp = results.event_data.str.extract(r'("correct"):(\w+)')
    results.insert(1,'attempts',atmp.loc[:,1])
    mapping={"true":1,"false":0}
    results['atmp_binary'] = results['attempts'].map(mapping)
    opt = results.groupby('installation_id').mean()
    opt['acgroup'] = 1
    opt.loc[opt[abs(opt['atmp_binary'] - 0.5) < 1e-9].index,'acgroup'] = 2
    opt.loc[opt[abs(opt['atmp_binary'] - 1) < 1e-9].index,'acgroup'] = 3
    opt.loc[opt[abs(opt['atmp_binary'] - 0) < 1e-9].index,'acgroup'] = 0
    
    return opt.acgroup.values[0]

In [205]:
def construct_train(train_ready,opty):
    # now we have accuracy group for all installation_id
    # start filling in other information
    #1, whether played same assessment before? if yes, assign times
    opty['pre_count'] = 0
    opty['pre_acgroup_mean'] = None
    inputdata = train_ready.loc[:,['installation_id','game_session','title','event_code','event_data']]
    for device in inputdata['installation_id'].unique():
        sample = inputdata[inputdata['installation_id'] == device]
        count = sample[sample['title'] == sample.iloc[-1].title].iloc[:-1].groupby('game_session').count().shape[0]
        opty.loc[opty[opty['installation_id'] == device].index,'pre_count'] = count
        #2, if played before, what is the acgroup?
        if count != 0:
            temp1 = sample[sample['title'] == sample.iloc[-1].title].iloc[:-1]
            opty.loc[opty[opty['installation_id'] == device].index,'pre_acgroup_mean'] = get_acgroup(temp1)
    
            
    
    
    
    return opty
        
        
        
        
    
    

In [206]:
construct_train(train_ready,opty)

1
     installation_id      game_session                       title  \
1143        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1144        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1145        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1146        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1147        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1148        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1149        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1150        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1151        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1152        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1153        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1154        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1155        0006a69f  77b8ee947eb84b4e  Bird Measurer (Assessment)   
1156        0006a6

,installation_id,acgroup,pre_count,pre_acgroup_mean
0,0006a69f,3,1,0
1,0006c192,2,0,None
2,00129856,3,0,None
3,001d0ed0,0,0,None
4,00225f67,0,0,None
5,00279ac5,0,0,None
6,002db7e3,0,0,None
7,003372b0,3,0,None
8,004c2091,0,0,None
9,00634433,0,0,None


In [160]:
sample = train_ready[train_ready['installation_id'] == '0006a69f']

In [188]:
temp1 = sample[sample['title'] == sample.iloc[-1].title].iloc[:-1]
_ = get_acgroup(temp1)
_



0